In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import helper
import matplotlib.pyplot as plt
import numpy as np
import copy
import random

In [3]:
data_dir='Apps/data/'

In [4]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))
                              ])
# Download and load the training data
trainset = datasets.MNIST(data_dir+'mnist', download=True, train=True, transform=transform)
testset = datasets.MNIST(data_dir+'mnist', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

0it [00:00, ?it/s]

100%|█████████▉| 9912320/9912422 [04:50<00:00, 24284.20it/s]

Extracting Apps/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to Apps/data/mnist/MNIST/raw



0it [00:00, ?it/s]


  0%|          | 0/28881 [00:00<?, ?it/s]
32768it [00:00, 46615.98it/s]                            

0it [00:00, ?it/s]

...

100%|█████████▉| 1646592/1648877 [00:51<00:00, 35976.88it/s]
1654784it [00:51, 32765.73it/s]                             

0it [00:00, ?it/s]

Extracting Apps/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to Apps/data/mnist/MNIST/raw




8192it [00:00, 16950.84it/s]            

Extracting Apps/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to Apps/data/mnist/MNIST/raw
Processing...
Done!




1654784it [01:08, 32765.73it/s]

# Prepare Dataset

## 1. same distribution with same size: split into 5 parts

In [11]:
datad={'trainset_'+str(i):[]   for i in range(10)}

for d in trainset:
    datad['trainset_'+str(d[1])].append(d)

In [16]:
dict1={}
for k in range(5):
    si=1084*k
    ei=1084*(k+1)
    dict1['trainset_1_'+str(k+1)] = []
    for i in range(10):
        dict1['trainset_1_'+str(k+1)].extend(datad['trainset_'+str(i)][si:ei])

## 2. different distribution with same size

In [45]:
dict2={'trainset_2_'+str(i):[] for i in range(1,6)}

ratio=0.8
big_n=int(5420*ratio)
small_n=round(5420*(1-ratio)/4)

last_index={i:0 for i in range(10)}

for i in range(1,6):
    for j in range(10):
        li=last_index[j]
        if j<2*i and j>=2*(i-1):
            dict2['trainset_2_'+str(i)].extend(datad['trainset_'+str(j)][li:li+big_n])
            last_index[j]=li+big_n
        else:
            dict2['trainset_2_'+str(i)].extend(datad['trainset_'+str(j)][li:li+small_n])
            last_index[j]=li+small_n


## 3. same distribution different size

In [47]:
dict3={'trainset_3_'+str(i):[] for i in range(1,6)}

for i in range(10):
  dict3['trainset_3_1'].extend(datad['trainset_'+str(i)][0:271*(2)])
  dict3['trainset_3_2'].extend(datad['trainset_'+str(i)][271*(2):271*(5)])
  dict3['trainset_3_3'].extend(datad['trainset_'+str(i)][271*(5):271*(9)])
  dict3['trainset_3_4'].extend(datad['trainset_'+str(i)][271*(9):271*(14)])
  dict3['trainset_3_5'].extend(datad['trainset_'+str(i)][271*(14):271*(20)])

for i in range(5):
  print(len(dict3['trainset_3_'+str(i+1)]))


5420
8130
10840
13550
16260


## 4. noisy label same size

In [50]:
dict4={}
for i in range(5):
  dict4["trainset_4_"+str(i+1)] = copy.deepcopy(dict1["trainset_1_"+str(i+1)]) 

In [51]:
percentageMislabel = {"trainset_4_1": 0, "trainset_4_2": 0.05, "trainset_4_3": 0.10, "trainset_4_4": 0.15, "trainset_4_5": 0.20}

for i in range(len(dict4)):
  trainset_4_i = "trainset_4_" + str(i+1)
  numMislabel = int(percentageMislabel[trainset_4_i] * len(dict4[trainset_4_i]))
  print("For trainset_" + str(i) + ", number of mislabelled data is: ", numMislabel)
  indexesMislabel = random.sample(range(len(dict4[trainset_4_i])), numMislabel) # generate a random sample of indices without replacement)
  print(len(indexesMislabel))

  for k in indexesMislabel: 
    trueLabel = dict4[trainset_4_i][i][1]
    possibleMislabel = [x for x in range(10) if x != trueLabel] # list of all the possible mislabels to choose from
    dict4[trainset_4_i][k] = (dict4[trainset_4_i][k][0], random.choice(possibleMislabel)) # change the label for the k-th entry 
  print("Done", i)
    

For trainset_0, number of mislabelled data is:  0
0
Done 0
For trainset_1, number of mislabelled data is:  542
542
Done 1
For trainset_2, number of mislabelled data is:  1084
1084
Done 2
For trainset_3, number of mislabelled data is:  1626
1626
Done 3
For trainset_4, number of mislabelled data is:  2168
2168
Done 4


## 5. noisy feature same size

In [52]:
dict5={}
for i in range(5):
  dict5["trainset_5_"+str(i+1)] = copy.deepcopy(dict1["trainset_1_"+str(i+1)]) 

In [53]:
percentageNoise = {"trainset_5_2": 0.25, "trainset_5_3": 0.5, "trainset_5_4": 0.75, "trainset_5_5": 1.0}

for i in range(1, len(dict5)):
  trainset_5_i = "trainset_5_" + str(i+1)
  for index in range(len(dict5[trainset_5_i])):
     dict5[trainset_5_i][index] = (dict5[trainset_5_i][index][0] + \
                                         torch.Tensor(np.random.normal(0.0, percentageNoise[trainset_5_i], (1,28,28))), \
                                         dict5[trainset_5_i][index][1])


# store datasets

In [54]:
import pickle

base_dir='Apps/data/OurMNIST/'

dictionary1File = open(base_dir+"dictionary1.out", "wb")
pickle.dump(dict1, dictionary1File)
dictionary1File.close()

dictionary2File = open(base_dir+"dictionary2.out", "wb")
pickle.dump(dict2, dictionary2File)
dictionary2File.close()

dictionary3File = open(base_dir+"dictionary3.out", "wb")
pickle.dump(dict3, dictionary3File)
dictionary3File.close()

dictionary4File = open(base_dir+"dictionary4.out", "wb")
pickle.dump(dict4, dictionary4File)
dictionary4File.close()

dictionary5File = open(base_dir+"dictionary5.out", "wb")
pickle.dump(dict5, dictionary5File)
dictionary5File.close()